# Example for steel production, equivalent to AMPL `steelT.mod`, using CPLEX
## Copyright (C) Princeton Consultants, 2017
### First, import the pandas library

In [1]:
import pandas as pd

### Read the data associated with the products

In [2]:
productDF = pd.read_excel("steelT.xlsx", index_col=0, skip_footer=18)
productDF

,rate,inv0,prodcost,invcost
Product,,,,
bands,220,10,10,2.5
coils,154,0,11,3.0


### Read the revenue and market data.  Note that we change the name of the index, because the Excel file has the name of the table, which is interpreted as the name of the index

In [3]:
revenue = pd.read_excel("steelT.xlsx", index_col=0, skiprows=6, skip_footer=12)
revenue.index.name = 'Product'
revenue

,1,2,3,4
Product,,,,
bands,25,26,27,27
coils,30,35,37,39


In [4]:
market = pd.read_excel("steelT.xlsx", index_col=0, skiprows=10, skip_footer=8)
market.index.name = 'Product'
market

,1,2,3,4
Product,,,,
bands,6000,6000,4000,6500
coils,4000,2500,3500,4200


### Read the available capacity per time period

In [5]:
avail = pd.read_excel("steelT.xlsx", index_col=0, skiprows=16, parse_cols=1)
avail

,avail
Time,
1,40
2,40
3,32
4,40


### Compute a tidy version of the revenue and market data

In [6]:
rmDF = pd.concat(
    [(pd.melt(market.reset_index(), id_vars=['Product'], value_name='market', var_name='T')
     .set_index(['Product','T']).astype('float64')),
     (pd.melt(revenue.reset_index(), id_vars=['Product'], value_name='revenue', var_name='T')
     .set_index(['Product','T']).astype('float64'))
    ], axis=1)
rmDF

,,market,revenue
Product,T,,
bands,1,6000.0,25.0
coils,1,4000.0,30.0
bands,2,6000.0,26.0
coils,2,2500.0,35.0
bands,3,4000.0,27.0
coils,3,3500.0,37.0
bands,4,6500.0,27.0
coils,4,4200.0,39.0


### Import the DoCplex library

In [7]:
from docplex.mp.model import Model

### Create a DoCplex modeling object

In [8]:
model = Model(name='steelT')

### Create a DataFrame containing the decision variables. Note that for the Sell variables, we can get the upper bounds right from the table

In [9]:
dvars = pd.DataFrame({'Make' : model.continuous_var_list(rmDF.index, name='Make'), 
                      'Inv' : model.continuous_var_list(rmDF.index, name='Inv'),
                      'Sell' : model.continuous_var_list(rmDF.index, ub=list(rmDF.market.values), name='Sell')},
                      index=rmDF.index)
dvars

,,Inv,Make,Sell
Product,T,,,
bands,1,"Inv_('bands', 1)","Make_('bands', 1)","Sell_('bands', 1)"
coils,1,"Inv_('coils', 1)","Make_('coils', 1)","Sell_('coils', 1)"
bands,2,"Inv_('bands', 2)","Make_('bands', 2)","Sell_('bands', 2)"
coils,2,"Inv_('coils', 2)","Make_('coils', 2)","Sell_('coils', 2)"
bands,3,"Inv_('bands', 3)","Make_('bands', 3)","Sell_('bands', 3)"
coils,3,"Inv_('coils', 3)","Make_('coils', 3)","Sell_('coils', 3)"
bands,4,"Inv_('bands', 4)","Make_('bands', 4)","Sell_('bands', 4)"
coils,4,"Inv_('coils', 4)","Make_('coils', 4)","Sell_('coils', 4)"


### Note that the types of the columns are objects.  In fact, they are decision variables

In [10]:
dvars.dtypes, type(dvars.iloc[0]['Make'])

(Inv     object
 Make    object
 Sell    object
 dtype: object, docplex.mp.linear.Var)

### Create a big DataFrame that merges the data with the decision variables, so that it is easier to write expressions

In [11]:
mainDF = (dvars.merge(rmDF, left_index=True, right_index=True, how='inner')
          .reset_index()
          .merge(productDF.reset_index(), left_on='Product', right_on='Product', how='left')
          .set_index(['Product','T'])
          )
mainDF


,,Inv,Make,Sell,market,revenue,rate,inv0,prodcost,invcost
Product,T,,,,,,,,,
bands,1,"Inv_('bands', 1)","Make_('bands', 1)","Sell_('bands', 1)",6000.0,25.0,220,10,10,2.5
coils,1,"Inv_('coils', 1)","Make_('coils', 1)","Sell_('coils', 1)",4000.0,30.0,154,0,11,3.0
bands,2,"Inv_('bands', 2)","Make_('bands', 2)","Sell_('bands', 2)",6000.0,26.0,220,10,10,2.5
coils,2,"Inv_('coils', 2)","Make_('coils', 2)","Sell_('coils', 2)",2500.0,35.0,154,0,11,3.0
bands,3,"Inv_('bands', 3)","Make_('bands', 3)","Sell_('bands', 3)",4000.0,27.0,220,10,10,2.5
coils,3,"Inv_('coils', 3)","Make_('coils', 3)","Sell_('coils', 3)",3500.0,37.0,154,0,11,3.0
bands,4,"Inv_('bands', 4)","Make_('bands', 4)","Sell_('bands', 4)",6500.0,27.0,220,10,10,2.5
coils,4,"Inv_('coils', 4)","Make_('coils', 4)","Sell_('coils', 4)",4200.0,39.0,154,0,11,3.0


### Set the objective function as
$$ \hbox{maximize}\quad\sum_{p,t}(revenue[p,t]*Sell[p,t]-prodcost[p]*Make[p,t]-invcost[p]*Inv[p,t])$$

In [12]:
model.maximize(model.sum(mainDF.revenue*mainDF.Sell-mainDF.prodcost*mainDF.Make-mainDF.invcost*mainDF.Inv))

### Illustrate how `groupby` can be used to create sums, and `itertuples` is used to iterate over the rows

In [13]:
[(t, tsum, avail) for (t, tsum, avail) in
 (pd.DataFrame((mainDF.Make/mainDF.rate).groupby(level='T').agg(model.sum), columns=['tsum'])
 .merge(avail, left_index=True, right_index=True)
 .itertuples()
 )
 ]

[(1, docplex.mp.LinearExpr(0.005Make_('bands', 1)+0.006Make_('coils', 1)), 40),
 (2, docplex.mp.LinearExpr(0.005Make_('bands', 2)+0.006Make_('coils', 2)), 40),
 (3, docplex.mp.LinearExpr(0.005Make_('bands', 3)+0.006Make_('coils', 3)), 32),
 (4, docplex.mp.LinearExpr(0.005Make_('bands', 4)+0.006Make_('coils', 4)), 40)]

### Now wrap the above code into the DoCplex `add_constraints` method, and add the constraints, naming them
$$\sum_p Make[p,t]*(1.0/rate[p]) \le avail[t], \qquad\forall p$$

In [14]:
model.add_constraints([(tsum <= avail, "Time_"+str(t)) for  (t, tsum, avail) in
 (pd.DataFrame((mainDF.Make/mainDF.rate).groupby(level='T').agg(model.sum), columns=['tsum'])
 .merge(avail, left_index=True, right_index=True)
 .itertuples()
 )
 ])

[docplex.mp.linear.LinearConstraint[Time_1](0.005Make_('bands', 1)+0.006Make_('coils', 1),LE,40),
 docplex.mp.linear.LinearConstraint[Time_2](0.005Make_('bands', 2)+0.006Make_('coils', 2),LE,40),
 docplex.mp.linear.LinearConstraint[Time_3](0.005Make_('bands', 3)+0.006Make_('coils', 3),LE,32),
 docplex.mp.linear.LinearConstraint[Time_4](0.005Make_('bands', 4)+0.006Make_('coils', 4),LE,40)]

### Use slicing to pick up the constraint for the first period, which has to refer to the initial inventory.  Show how the slice works.

In [15]:
idx = pd.IndexSlice
mainDF.sort_index(inplace=True)
mainDF.loc[idx[:,1],['Make','inv0','Sell','Inv']].reset_index()

,Product,T,Make,inv0,Sell,Inv
0,bands,1,"Make_('bands', 1)",10,"Sell_('bands', 1)","Inv_('bands', 1)"
1,coils,1,"Make_('coils', 1)",0,"Sell_('coils', 1)","Inv_('coils', 1)"


### Add the constraint for the first time period
$$Make[p,1] + inv0[p] = Sell[p,1]+Inv[p,1]\qquad\forall p$$

In [16]:
model.add_constraints([(Make + inv0 == Sell + Inv, 'Balance_'+Product+'_'+str(t))
                      for ((Product, t), Make, inv0, Sell, Inv) in
                      mainDF.loc[idx[:,1],['Make','inv0','Sell','Inv']].itertuples()])

[docplex.mp.linear.LinearConstraint[Balance_bands_1](Make_('bands', 1)+10,EQ,Sell_('bands', 1)+Inv_('bands', 1)),
 docplex.mp.linear.LinearConstraint[Balance_coils_1](Make_('coils', 1),EQ,Sell_('coils', 1)+Inv_('coils', 1))]

### Show how slicing can pick up the relevant variables for the constraints for period 2 onwards

In [17]:
mainDF.loc[idx[:,2:],['Make','Sell','Inv']]

Make               Sell               Inv
Product T                                                        
bands   2  Make_('bands', 2)  Sell_('bands', 2)  Inv_('bands', 2)
        3  Make_('bands', 3)  Sell_('bands', 3)  Inv_('bands', 3)
        4  Make_('bands', 4)  Sell_('bands', 4)  Inv_('bands', 4)
coils   2  Make_('coils', 2)  Sell_('coils', 2)  Inv_('coils', 2)
        3  Make_('coils', 3)  Sell_('coils', 3)  Inv_('coils', 3)
        4  Make_('coils', 4)  Sell_('coils', 4)  Inv_('coils', 4)

### Now add the balance constraints. Note the use of `.loc` to refer to the earlier period
$$Make[p,1] + Inv[p,t-1] = Sell[p,1]+Inv[p,1]\qquad\forall p, t\ge2$$

In [18]:
model.add_constraints([(Make + mainDF.Inv.loc[Product, t-1] == Sell + Inv, 'Balance_'+Product+'_'+str(t))
                       for ((Product, t), Make, Sell, Inv) in
                       mainDF.loc[idx[:,2:],['Make','Sell','Inv']].itertuples()])

[docplex.mp.linear.LinearConstraint[Balance_bands_2](Make_('bands', 2)+Inv_('bands', 1),EQ,Sell_('bands', 2)+Inv_('bands', 2)),
 docplex.mp.linear.LinearConstraint[Balance_bands_3](Make_('bands', 3)+Inv_('bands', 2),EQ,Sell_('bands', 3)+Inv_('bands', 3)),
 docplex.mp.linear.LinearConstraint[Balance_bands_4](Make_('bands', 4)+Inv_('bands', 3),EQ,Sell_('bands', 4)+Inv_('bands', 4)),
 docplex.mp.linear.LinearConstraint[Balance_coils_2](Make_('coils', 2)+Inv_('coils', 1),EQ,Sell_('coils', 2)+Inv_('coils', 2)),
 docplex.mp.linear.LinearConstraint[Balance_coils_3](Make_('coils', 3)+Inv_('coils', 2),EQ,Sell_('coils', 3)+Inv_('coils', 3)),
 docplex.mp.linear.LinearConstraint[Balance_coils_4](Make_('coils', 4)+Inv_('coils', 3),EQ,Sell_('coils', 4)+Inv_('coils', 4))]

### Time to solve

In [19]:
model.solve()

docplex.mp.solution.SolveSolution(obj=562473,values={Make_('coils', 4):4..

### Grab the solution and put the solution in a DataFrame

In [20]:
soln = pd.DataFrame({'Make' : [x.solution_value for x in mainDF.Make],
                     'Sell' : [x.solution_value for x in mainDF.Sell],
                     'Inv' : [x.solution_value for x in mainDF.Inv]},
                    index = mainDF.index)
soln

Inv    Make    Sell
Product T                       
bands   1    0.0  5990.0  6000.0
        2    0.0  6000.0  6000.0
        3    0.0  2040.0  2040.0
        4    0.0  2800.0  2800.0
coils   1  540.0  1967.0  1427.0
        2    0.0  1960.0  2500.0
        3    0.0  3500.0  3500.0
        4    0.0  4200.0  4200.0